In [ ]:
#!/usr/bin/env python3
"""
Stock Analysis Module for Indian Market Research System v1.0
Created: January 2022
Focus: Indian Stock Market (NSE/BSE)
Compatible with Google Colab
"""

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib for better plots
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

class IndianStockAnalyzer:
    """
    Indian Stock Market Analysis System
    Focuses on NSE/BSE listed companies
    """
    
    def __init__(self):
        # Top Indian stocks by market cap (2022)
        self.nifty_50_symbols = [
            'RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS', 'INFY.NS',
            'ICICIBANK.NS', 'HDFC.NS', 'ITC.NS', 'SBIN.NS', 'BHARTIARTL.NS',
            'ASIANPAINT.NS', 'MARUTI.NS', 'BAJFINANCE.NS', 'LT.NS', 'HCLTECH.NS',
            'AXISBANK.NS', 'WIPRO.NS', 'ULTRACEMCO.NS', 'DMART.NS', 'NESTLEIND.NS',
            'ONGC.NS', 'TITAN.NS', 'SUNPHARMA.NS', 'KOTAKBANK.NS', 'POWERGRID.NS',
            'NTPC.NS', 'TECHM.NS', 'M&M.NS', 'TATAMOTORS.NS', 'BAJAJ-AUTO.NS',
            'GRASIM.NS', 'JSWSTEEL.NS', 'HEROMOTOCO.NS', 'CIPLA.NS', 'COALINDIA.NS',
            'UPL.NS', 'DRREDDY.NS', 'BRITANNIA.NS', 'EICHERMOT.NS', 'BPCL.NS',
            'TATACONSUM.NS', 'HINDALCO.NS', 'ADANIPORTS.NS', 'TATASTEEL.NS', 
            'INDUSINDBK.NS', 'DIVISLAB.NS', 'BAJAJFINSV.NS', 'SBILIFE.NS', 'HDFCLIFE.NS'
        ]
        
        # Sector categorization
        self.sectors = {
            'Banking & Finance': ['HDFCBANK.NS', 'ICICIBANK.NS', 'SBIN.NS', 'AXISBANK.NS', 
                                'KOTAKBANK.NS', 'INDUSINDBK.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS'],
            'Information Technology': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS', 'TECHM.NS'],
            'Oil & Gas': ['RELIANCE.NS', 'ONGC.NS', 'BPCL.NS'],
            'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'NESTLEIND.NS', 'BRITANNIA.NS', 'TATACONSUM.NS'],
            'Automobiles': ['MARUTI.NS', 'TATAMOTORS.NS', 'BAJAJ-AUTO.NS', 'HEROMOTOCO.NS', 'EICHERMOT.NS'],
            'Pharmaceuticals': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS', 'DIVISLAB.NS'],
            'Metals & Mining': ['JSWSTEEL.NS', 'TATASTEEL.NS', 'HINDALCO.NS', 'COALINDIA.NS'],
            'Consumer Durables': ['ASIANPAINT.NS', 'TITAN.NS', 'DMART.NS'],
            'Infrastructure': ['LT.NS', 'POWERGRID.NS', 'NTPC.NS', 'ADANIPORTS.NS'],
            'Telecom': ['BHARTIARTL.NS']
        }
        
        self.indices = {
            '^NSEI': 'NIFTY 50',
            '^BSESN': 'SENSEX',
            '^NSEBANK': 'BANK NIFTY',
            '^NSMIDCP': 'NIFTY MIDCAP'
        }
    
    def fetch_stock_data(self, symbol, period='1y', interval='1d'):
        """
        Fetch stock data from Yahoo Finance
        
        Args:
            symbol (str): Stock symbol (e.g., 'RELIANCE.NS')
            period (str): Data period ('1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max')
            interval (str): Data interval ('1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo')
        
        Returns:
            pandas.DataFrame: Stock data with OHLCV
        """
        try:
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period, interval=interval)
            
            if data.empty:
                print(f"No data found for symbol: {symbol}")
                return None
                
            # Add additional columns
            data['Returns'] = data['Close'].pct_change()
            data['Volatility'] = data['Returns'].rolling(window=20).std() * np.sqrt(252)
            
            return data
        except Exception as e:
            print(f"Error fetching data for {symbol}: {str(e)}")
            return None
    
    def calculate_technical_indicators(self, data):
        """
        Calculate technical indicators for stock analysis
        """
        df = data.copy()
        
        # Simple Moving Averages
        df['SMA_20'] = df['Close'].rolling(window=20).mean()
        df['SMA_50'] = df['Close'].rolling(window=50).mean()
        df['SMA_200'] = df['Close'].rolling(window=200).mean()
        
        # Exponential Moving Averages
        df['EMA_12'] = df['Close'].ewm(span=12).mean()
        df['EMA_26'] = df['Close'].ewm(span=26).mean()
        
        # MACD
        df['MACD'] = df['EMA_12'] - df['EMA_26']
        df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
        df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
        
        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # Bollinger Bands
        df['BB_Middle'] = df['Close'].rolling(window=20).mean()
        bb_std = df['Close'].rolling(window=20).std()
        df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
        df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
        df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
        df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
        
        # Average True Range (ATR)
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)
        df['ATR'] = true_range.rolling(14).mean()
        
        # Volume indicators
        df['Volume_SMA'] = df['Volume'].rolling(window=20).mean()
        df['Volume_Ratio'] = df['Volume'] / df['Volume_SMA']
        
        # Support and Resistance levels (simplified)
        df['Resistance'] = df['High'].rolling(window=20).max()
        df['Support'] = df['Low'].rolling(window=20).min()
        
        return df
    
    def analyze_single_stock(self, symbol, period='1y'):
        """
        Comprehensive analysis of a single stock
        """
        print(f"\n{'='*50}")
        print(f"ANALYZING: {symbol}")
        print(f"{'='*50}")
        
        data = self.fetch_stock_data(symbol, period)
        if data is None:
            return None
        
        # Calculate technical indicators
        data = self.calculate_technical_indicators(data)
        
        # Basic statistics
        latest_price = data['Close'].iloc[-1]
        price_change = data['Returns'].iloc[-1] * 100
        volatility = data['Volatility'].iloc[-1] * 100
        
        # Performance metrics
        total_return = ((data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1) * 100
        max_price = data['Close'].max()
        min_price = data['Close'].min()
        
        # Technical signals
        current_rsi = data['RSI'].iloc[-1]
        current_macd = data['MACD'].iloc[-1]
        current_bb_position = data['BB_Position'].iloc[-1]
        
        # Volume analysis
        avg_volume = data['Volume'].mean()
        current_volume = data['Volume'].iloc[-1]
        volume_surge = (current_volume / avg_volume) > 1.5
        
        # Trend analysis
        sma_20 = data['SMA_20'].iloc[-1]
        sma_50 = data['SMA_50'].iloc[-1]
        sma_200 = data['SMA_200'].iloc[-1]
        
        trend = "BULLISH" if latest_price > sma_20 > sma_50 > sma_200 else \
                "BEARISH" if latest_price < sma_20 < sma_50 < sma_200 else "SIDEWAYS"
        
        # Print analysis
        print(f"Latest Price: ₹{latest_price:.2f}")
        print(f"Price Change: {price_change:.2f}%")
        print(f"Total Return ({period}): {total_return:.2f}%")
        print(f"Volatility: {volatility:.2f}%")
        print(f"Trend: {trend}")
        print(f"RSI: {current_rsi:.2f} ({'Overbought' if current_rsi > 70 else 'Oversold' if current_rsi < 30 else 'Neutral'})")
        print(f"MACD: {current_macd:.4f}")
        print(f"Volume Surge: {'Yes' if volume_surge else 'No'}")
        print(f"Bollinger Band Position: {current_bb_position:.2f} ({'Upper Band' if current_bb_position > 0.8 else 'Lower Band' if current_bb_position < 0.2 else 'Middle'})")
        
        return {
            'symbol': symbol,
            'data': data,
            'metrics': {
                'latest_price': latest_price,
                'price_change': price_change,
                'total_return': total_return,
                'volatility': volatility,
                'trend': trend,
                'rsi': current_rsi,
                'macd': current_macd,
                'volume_surge': volume_surge,
                'bb_position': current_bb_position
            }
        }
    
    def plot_stock_analysis(self, symbol, period='6mo'):
        """
        Create comprehensive stock analysis charts
        """
        data = self.fetch_stock_data(symbol, period)
        if data is None:
            return
        
        data = self.calculate_technical_indicators(data)
        
        # Create subplots
        fig, axes = plt.subplots(4, 1, figsize=(15, 16))
        fig.suptitle(f'{symbol} - Comprehensive Technical Analysis', fontsize=16, fontweight='bold')
        
        # Price and Moving Averages
        axes[0].plot(data.index, data['Close'], label='Close Price', linewidth=2, color='black')
        axes[0].plot(data.index, data['SMA_20'], label='SMA 20', alpha=0.7, color='blue')
        axes[0].plot(data.index, data['SMA_50'], label='SMA 50', alpha=0.7, color='orange')
        axes[0].plot(data.index, data['BB_Upper'], label='BB Upper', alpha=0.5, color='red', linestyle='--')
        axes[0].plot(data.index, data['BB_Lower'], label='BB Lower', alpha=0.5, color='green', linestyle='--')
        axes[0].fill_between(data.index, data['BB_Upper'], data['BB_Lower'], alpha=0.1, color='gray')
        axes[0].set_title('Price Action with Moving Averages & Bollinger Bands')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Volume
        colors = ['green' if x > 0 else 'red' for x in data['Returns']]
        axes[1].bar(data.index, data['Volume'], color=colors, alpha=0.7)
        axes[1].plot(data.index, data['Volume_SMA'], color='blue', label='Volume SMA')
        axes[1].set_title('Volume Analysis')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        # RSI
        axes[2].plot(data.index, data['RSI'], color='purple', linewidth=2)
        axes[2].axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
        axes[2].axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
        axes[2].fill_between(data.index, 70, 100, alpha=0.1, color='red')
        axes[2].fill_between(data.index, 0, 30, alpha=0.1, color='green')
        axes[2].set_title('RSI (Relative Strength Index)')
        axes[2].set_ylim(0, 100)
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)
        
        # MACD
        axes[3].plot(data.index, data['MACD'], label='MACD', color='blue')
        axes[3].plot(data.index, data['MACD_Signal'], label='Signal', color='red')
        axes[3].bar(data.index, data['MACD_Histogram'], label='Histogram', alpha=0.6, color='gray')
        axes[3].axhline(y=0, color='black', linestyle='-', alpha=0.5)
        axes[3].set_title('MACD (Moving Average Convergence Divergence)')
        axes[3].legend()
        axes[3].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def sector_analysis(self, period='6mo'):
        """
        Analyze performance by sector
        """
        print(f"\n{'='*60}")
        print("SECTOR PERFORMANCE ANALYSIS")
        print(f"{'='*60}")
        
        sector_performance = {}
        
        for sector, stocks in self.sectors.items():
            print(f"\nAnalyzing {sector}...")
            sector_returns = []
            
            for stock in stocks:
                data = self.fetch_stock_data(stock, period)
                if data is not None and len(data) > 1:
                    total_return = ((data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1) * 100
                    sector_returns.append(total_return)
            
            if sector_returns:
                avg_return = np.mean(sector_returns)
                sector_performance[sector] = {
                    'avg_return': avg_return,
                    'stocks_analyzed': len(sector_returns),
                    'individual_returns': sector_returns
                }
                print(f"{sector}: {avg_return:.2f}% (analyzed {len(sector_returns)} stocks)")
        
        # Plot sector performance
        sectors = list(sector_performance.keys())
        returns = [sector_performance[sector]['avg_return'] for sector in sectors]
        
        plt.figure(figsize=(12, 8))
        colors = ['green' if r > 0 else 'red' for r in returns]
        bars = plt.bar(sectors, returns, color=colors, alpha=0.7)
        plt.title(f'Sector Performance Comparison ({period})', fontsize=14, fontweight='bold')
        plt.ylabel('Average Return (%)')
        plt.xticks(rotation=45, ha='right')
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        plt.grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar, value in zip(bars, returns):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + (0.5 if value > 0 else -1.5), 
                    f'{value:.1f}%', ha='center', va='bottom' if value > 0 else 'top', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        return sector_performance
    
    def market_overview(self, period='6mo'):
        """
        Overall market analysis using major indices
        """
        print(f"\n{'='*60}")
        print("INDIAN MARKET OVERVIEW")
        print(f"{'='*60}")
        
        market_data = {}
        
        for symbol, name in self.indices.items():
            data = self.fetch_stock_data(symbol, period)
            if data is not None:
                latest_price = data['Close'].iloc[-1]
                total_return = ((data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1) * 100
                volatility = data['Returns'].std() * np.sqrt(252) * 100
                
                market_data[name] = {
                    'price': latest_price,
                    'return': total_return,
                    'volatility': volatility
                }
                
                print(f"{name}: {latest_price:.2f} ({total_return:+.2f}%) | Vol: {volatility:.2f}%")
        
        return market_data
    
    def generate_stock_report(self, symbols=None, period='6mo'):
        """
        Generate comprehensive report for selected stocks
        """
        if symbols is None:
            symbols = self.nifty_50_symbols[:10]  # Top 10 for quick analysis
        
        print(f"\n{'='*80}")
        print(f"COMPREHENSIVE STOCK ANALYSIS REPORT - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"{'='*80}")
        
        # Market Overview
        self.market_overview(period)
        
        # Individual stock analysis
        stock_results = []
        for symbol in symbols:
            result = self.analyze_single_stock(symbol, period)
            if result:
                stock_results.append(result)
        
        # Sector Analysis
        sector_perf = self.sector_analysis(period)
        
        # Summary insights
        print(f"\n{'='*60}")
        print("KEY INSIGHTS & RECOMMENDATIONS")
        print(f"{'='*60}")
        
        if stock_results:
            # Best and worst performers
            best_performer = max(stock_results, key=lambda x: x['metrics']['total_return'])
            worst_performer = min(stock_results, key=lambda x: x['metrics']['total_return'])
            
            print(f"🏆 Best Performer: {best_performer['symbol']} (+{best_performer['metrics']['total_return']:.2f}%)")
            print(f"📉 Worst Performer: {worst_performer['symbol']} ({worst_performer['metrics']['total_return']:.2f}%)")
            
            # Overbought/Oversold stocks
            overbought = [s for s in stock_results if s['metrics']['rsi'] > 70]
            oversold = [s for s in stock_results if s['metrics']['rsi'] < 30]
            
            if overbought:
                print(f"⚠️  Overbought Stocks: {', '.join([s['symbol'] for s in overbought])}")
            if oversold:
                print(f"💡 Oversold Opportunities: {', '.join([s['symbol'] for s in oversold])}")
        
        print(f"\nReport generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*80)


def main():
    """
    Main execution function for Google Colab
    """
    # Initialize analyzer
    analyzer = IndianStockAnalyzer()
    
    # Example usage - uncomment the sections you want to run
    
    # 1. Analyze a single stock
    print("1. SINGLE STOCK ANALYSIS")
    analyzer.analyze_single_stock('RELIANCE.NS', '1y')
    analyzer.plot_stock_analysis('RELIANCE.NS', '6mo')
    
    # 2. Sector analysis
    print("\n2. SECTOR ANALYSIS")
    analyzer.sector_analysis('6mo')
    
    # 3. Market overview
    print("\n3. MARKET OVERVIEW")
    analyzer.market_overview('1y')
    
    # 4. Comprehensive report
    print("\n4. COMPREHENSIVE REPORT")
    top_stocks = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'INFY.NS', 'HINDUNILVR.NS']
    analyzer.generate_stock_report(top_stocks, '6mo')

if __name__ == "__main__":
    main()